# Autoencoder
#*Jan Miksa*
based on: https://pytorch.org/tutorials/beginner/basics/intro.html

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import math
import sys

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

batch_size = 64
image_size = 28
channels = 1

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
class Encoder(nn.Module):
    def __init__(self, kernel_size):
        super(Encoder, self).__init__()
        self.kernel_size = kernel_size
        self.build()

    def build(self):
        #x = 1
        #i = 0
        #while x < image_size:
        #    self.conv = nn.Conv2d((channels*self.kernel_size**2)**i, (channels*self.kernel_size**2)**(i+1), self.kernel_size)
        #    x = (channels*self.kernel_size**2)**(i+1)
        #    i = i+1
        self.conv1 = nn.Conv2d(1, 9, 3)
        self.conv2 = nn.Conv2d(9, 81, 3)

    def forward(self, x):
        #for l in self.modules():
        #    x = l(x)
        x = self.conv1(x)
        x = self.conv2(x)
        return x

class Decoder(nn.Module):
    def __init__(self, kernel_size):
        super(Decoder, self).__init__()
        self.kernel_size = kernel_size
        self.build()

    def build(self):
        #x = 28
        #i = 0
        #while x > channels:
        #    self.conv = nn.Conv2d(math.ceil(image_size/(channels*self.kernel_size**2)**i), math.ceil(image_size/(channels*self.kernel_size**2)**(i+1)), self.kernel_size)
        #    x =  math.ceil(image_size/(channels*self.kernel_size**2)**(i+1))
        #    i = i+1
        #ConvTranspose2d
        self.conv1 = nn.ConvTranspose2d(81, 9, 3)
        self.conv2 = nn.ConvTranspose2d(9, 1, 3)
        self.conv3 = nn.ConvTranspose2d(2, 1, 1)

    def forward(self, x):
        #for l in self.modules():
        #    x = l(x)
        x = self.conv1(x)
        x = self.conv2(x)
        return x

class Autoencoder(nn.Module):
    def __init__(self, in_channels, kernel_size):
        super(Autoencoder, self).__init__()
        self.enc = Encoder(3)
        #flatten
        #reshape
        self.dec = Decoder(3)

    def forward(self, x):
        x = self.enc.forward(x)
        x = self.dec.forward(x)
        return x

model = Autoencoder(1, 3).to(device)

In [ ]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)

        res = model(X)
        loss = loss_fn(res, X)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device)
            res = model(X)
            test_loss += loss_fn(res, X).item()
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

In [ ]:
for i in model.modules():
  print(i)


Autoencoder(
  (enc): Encoder(
    (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  )
  (dec): Decoder(
    (conv1): Conv2d(6, 3, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1))
  )
)
Encoder(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
)
Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
Decoder(
  (conv1): Conv2d(6, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1))
)
Conv2d(6, 3, kernel_size=(3, 3), stride=(1, 1))
Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1))


In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
#Porównanie przed i po obrazków

Epoch 1
-------------------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([64, 1, 28, 28])) that is different to the input size (torch.Size([64, 1, 20, 20])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


RuntimeError: ignored